# CNNのハイパーパラメータチューニング

とりあえず、隠れ層の数は3層として、もっとも精度の高いモデルのハイパーパラメータを求めることを目的とする。

### ライブラリのインポート

In [43]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

### データの準備

In [44]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#### 形の確認

In [45]:
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

(60000, 28, 28, 1)
(60000, 10)
(10000, 28, 28, 1)
(10000, 10)


### モデルの作成

In [46]:
def build_model(activation , optimizer):
    model = Sequential()

    model.add(Conv2D(input_shape=(28, 28, 1), filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1)))

    model.add(Flatten())
    model.add(Dense(256, activation=activation, input_dim=784))
    model.add(Dense(128, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(10, activation='softmax'))

    # model.add(Flatten())
    # model.add(Dense(256, input_dim=784))
    # model.add(Activation(activation))
    # model.add(Dense(128))
    # model.add(Activation(activation))
    # model.add(Dense(128))
    # model.add(Activation(activation))
    # model.add(Dense(10))
    # model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

### グリッドサーチを行う

In [47]:
# ハイパーパラメータを準備
activation = ['relu', 'sigmoid']
optimizer = ['adam', 'sgd']
nb_epoch = [10, 20]
batch_size = [64, 128, 256]

#グリッドサーチ対象のハイパーパラメーターを辞書型にまとめる
param_grid = dict(activation=activation, optimizer=optimizer,nb_epoch=nb_epoch, batch_size=batch_size)

#モデルを作成
model = KerasRegressor(build_fn = build_model, verbose=True)

#グリッドサーチの実行
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

938/938 [==============================] - 10s 11ms/step - loss: 0.7552 - accuracy: 0.9267


### 結果の確認

In [48]:
print(grid_result.best_params_)

{'activation': 'relu', 'batch_size': 64, 'nb_epoch': 20, 'optimizer': 'adam'}


# 求めた最適ハイパーパラメータでモデルを作成

In [51]:
model = build_model(activation='relu', optimizer='adam')

history = model.fit(X_train, y_train, epochs=20, batch_size=64)

Epoch 1/20
938/938 [==============================] - 10s 11ms/step - loss: 1.7084 - accuracy: 0.9185
Epoch 2/20
938/938 [==============================] - 10s 11ms/step - loss: 0.1296 - accuracy: 0.9684
Epoch 3/20
938/938 [==============================] - 10s 11ms/step - loss: 0.1089 - accuracy: 0.9736
Epoch 4/20
938/938 [==============================] - 10s 11ms/step - loss: 0.0746 - accuracy: 0.9807
Epoch 5/20
938/938 [==============================] - 10s 11ms/step - loss: 0.0802 - accuracy: 0.9793
Epoch 6/20
938/938 [==============================] - 10s 11ms/step - loss: 0.0726 - accuracy: 0.9815
Epoch 7/20
938/938 [==============================] - 10s 11ms/step - loss: 0.0786 - accuracy: 0.9809
Epoch 8/20
938/938 [==============================] - 10s 11ms/step - loss: 0.0561 - accuracy: 0.9853
Epoch 9/20
938/938 [==============================] - 10s 11ms/step - loss: 0.0621 - accuracy: 0.9842
Epoch 10/20
938/938 [==============================] - 10s 11ms/step - loss: 0.052

In [52]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0661 - accuracy: 0.9872


[0.06611067801713943, 0.9872000217437744]